In [ ]:
import pandas as pd

# Load data
sensor_df = pd.read_csv("sensor_semantics.csv")
distance_df = pd.read_csv("sensor_semantics_distances.csv")

# Define Turtle prefixes
prefixes = """@prefix : <http://example.org/traffic#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
"""

ttl = prefixes

# Sanitize string literals before inserting into TTL
def escape_ttl_string(val):
    return str(val).replace('"', '\\"').replace('\n', ' ').strip()

def make_uri_safe(val):
    return str(val).replace(" ", "_").replace("/", "_").replace(":", "_")


# Add TrafficSensor and RoadSegment instances with full property mappings
for _, row in sensor_df.iterrows():
    sensor_uri = f":sensor_{row['sensor_id']}"
    road_uri = f":road_{row['matched_road_id']}"
    
    ttl += f"{sensor_uri} rdf:type :TrafficSensor ;\n"
    ttl += f"    :installedOn {road_uri} .\n\n"

    ttl += f"{road_uri} rdf:type :RoadSegment ;\n"
    ttl += f"    :roadType \"{escape_ttl_string(row['matched_road_type'])}\"^^xsd:string ;\n"
    ttl += f"    :maxSpeed \"{row['max_speed']}\"^^xsd:float ;\n"
    ttl += f"    :numberOfLanes \"{int(row['lanes'])}\"^^xsd:integer .\n\n"

# Add sensor-to-sensor networkDistance resources
for _, row in distance_df.iterrows():
    from_sensor_id = int(row['from'])
    to_sensor_id = int(row['to'])
    cost = row['cost']

    from_sensor = f":sensor_{from_sensor_id}"
    to_sensor = f":sensor_{to_sensor_id}"
    dist_uri = f":dist_{from_sensor_id}_{to_sensor_id}"

    ttl += f"{dist_uri} a :NetworkDistance ;\n"
    ttl += f"    :fromSensor {from_sensor} ;\n"
    ttl += f"    :toSensor {to_sensor} ;\n"
    ttl += f"    :distanceValue \"{cost}\"^^xsd:float ;\n"
    ttl += f"    :distanceUnit \"meters\"^^xsd:string .\n\n"


# Write to TTL file
with open("LA_traffic_KG.ttl", "w") as f:
    f.write(ttl)

print("TTL file generated: LA_traffic_KG.ttl")


✅ TTL file generated: LA_traffic_KG.ttl
